<a href="https://colab.research.google.com/github/muzahid120/Deeplearning/blob/Python-project/CNN/transfer_learning/data%20augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator

In [17]:
import os
import shutil

# Define paths
kaggle_dir = os.path.expanduser('~/.kaggle')
kaggle_json_src = 'kaggle.json'
kaggle_json_dest = os.path.join(kaggle_dir, 'kaggle.json')

# Create the .kaggle directory if it doesn't exist
if not os.path.exists(kaggle_dir):
    os.makedirs(kaggle_dir)

# Move the kaggle.json file to the .kaggle directory
shutil.move(kaggle_json_src, kaggle_json_dest)

# Set permissions for kaggle.json file
os.chmod(kaggle_json_dest, 0o600)

In [18]:
# Specify the directory path
directory_path = os.path.expanduser('~/.kaggle')

# Create the directory if it doesn't exist
os.makedirs(directory_path, exist_ok=True)


In [4]:
! kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:16<00:00, 83.2MB/s]
100% 1.06G/1.06G [00:16<00:00, 70.2MB/s]


In [5]:
import zipfile
def unzip_file(zip_file_path, extract_to_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
        zip_ref.close()


In [6]:
zip_file_path = '/content/dogs-vs-cats.zip'
extract_to_path = '/content'

unzip_file(zip_file_path, extract_to_path)

In [19]:
batch_size=16
# test_data_genoratore
train_gen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True

)

#test_data_generatore

test_gen=ImageDataGenerator(
    rescale=1./255
)


# train data generatore from directory
train_gen_data=train_gen.flow_from_directory('/content/train',
                              target_size=(224,224),
                              batch_size=batch_size,
                              class_mode='binary')
# test data genoratore from directory

test_gen_data=test_gen.flow_from_directory('/content/test',
                                           target_size=(224,224),
                                           batch_size=batch_size,
                                           class_mode='binary')


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [20]:
conv_base=VGG16(include_top=False,
                weights='imagenet',
                input_shape=(224,224,3))

In [9]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [21]:
conv_base.trainable=False
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [22]:
model=Sequential()
model.add(conv_base)
model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               6422784   
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21137729 (80.63 MB)
Trainable params: 6423041 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [24]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [25]:
history=model.fit_generator(train_gen_data,
                            steps_per_epoch=20000/224,
                            epochs=10,
                            validation_data=test_gen_data,
                            validation_steps=5000/224)

<ipython-input-25-eb9a99414d54>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_gen_data,


Epoch 1/10
 6/89 [=>............................] - ETA: 11:40 - loss: 9.5742 - accuracy: 0.5417 

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

NameError: name 'history' is not defined